<span style="font-size: 250%"> Projet 9 : Une étude de marché de poulet avec Python </span>

1. La classification ascendante hiérarchique,avec un dendrogramme comme visualisation
2. Utiliser la méthode des k-means, afin d’affiner l’analyse et comparer les résultats des deux méthodes de clustering.
3. Analyser les centroïdes de tes classes
4. Réaliser une ACP afin de visualiser les résultats de ton analyse, comprendre les groupes, les liens entre les variables, les liens entre les individus...

# Les enjeux

## Mise en situation

L'entreprise d'agroalimentaire souhaite se développer à l'international.<br> 
Elle est spécialisée dans le poulet !<br>

L'international, oui, mais pour l'instant, le champ des possibles est bien large : aucun pays particulier ni aucun continent n'est pour le moment choisi. Tous les pays sont envisageables !<br>

Votre objectif sera d'aider à cibler plus particulièrement certains pays, dans le but d'approfondir ensuite l'étude de marché. Plus particulièrement, l'idéal serait de produire des "groupes" de pays, plus ou moins gros, dont on connaît les caractéristiques.<br>

Dans un premier temps, la stratégie est plutôt d'exporter les produits plutôt que de produire sur place, c'est-à-dire dans le(s) nouveau(x) pays ciblé(s).<br>

## Les données

Pour ce projet, à nous de retrouver les données utiles pour une étude de marché fiable. Nous nous replongeons sur le site de la FAO car regorge de nombre données sur les bilans alimentatires(important pour cette étude).

 #  Importation des librairies 

In [1]:
import pandas as pd # (version pandas 1.4.2)
import numpy as np  # (version numpy 1.22.4)
import seaborn as sns # (version seaborn 0.11.2)
import scipy.stats as st
import datetime 
sns.set_palette("colorblind")

import matplotlib.pyplot as plt
import plotly.graph_objects as go
#import lux

from scipy.stats import shapiro

from fonction import testerCle
from fonction import moving_average

colors = ["#b2eca7","#ffcc97","#676283","#72bbff","#d2a0f2","#0abdc0"]

# Pour éviter d'avoir les messages warning
import warnings
warnings.filterwarnings('ignore')

# Importation des données 

In [2]:
# importation des données

df_dispo_alim = pd.read_csv('DisponibiliteAlimentaire_2017.csv') # données sur la population de chaque pays
df_population = pd.read_csv('Population_2000_2018.csv') # données sur la disponibilité alimentaire des pays
df_economie = pd.read_csv('Indicateur_macro1.csv') # données sur l'activité économique des pays
df_reserve_poulet = pd.read_csv('cout_production_poulet.csv') # données coût de production du poulet dans les pays
df_economy = pd.read_excel('economy.xlsx')
df_stabilite = pd.read_excel('stabilité_data.xlsx')

# Analyse des dataframe et les nettroyer

## Données Population

### Lecture du données

In [3]:
df_population.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


In [4]:
df_population.describe()

,Code zone,Code Élément,Code Produit,Code année,Année,Valeur
count,4411.000000,4411.0,4411.0,4411.000000,4411.000000,4.411000e+03
mean,132.202902,511.0,3010.0,2009.068238,2009.068238,2.963004e+04
std,75.854840,0.0,0.0,5.481539,5.481539,1.238029e+05
min,1.000000,511.0,3010.0,2000.000000,2000.000000,7.850000e-01
25%,68.000000,511.0,3010.0,2004.000000,2004.000000,3.921890e+02
50%,132.000000,511.0,3010.0,2009.000000,2009.000000,4.764741e+03
75%,195.000000,511.0,3010.0,2014.000000,2014.000000,1.821548e+04
max,299.000000,511.0,3010.0,2018.000000,2018.000000,1.427648e+06


In [5]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            4411 non-null   object 
 1   Domaine                 4411 non-null   object 
 2   Code zone               4411 non-null   int64  
 3   Zone                    4411 non-null   object 
 4   Code Élément            4411 non-null   int64  
 5   Élément                 4411 non-null   object 
 6   Code Produit            4411 non-null   int64  
 7   Produit                 4411 non-null   object 
 8   Code année              4411 non-null   int64  
 9   Année                   4411 non-null   int64  
 10  Unité                   4411 non-null   object 
 11  Valeur                  4411 non-null   float64
 12  Symbole                 4411 non-null   object 
 13  Description du Symbole  4411 non-null   object 
 14  Note                    258 non-null    

In [6]:
df_population.shape

(4411, 15)

### Nettoyage 

In [7]:
# Changer l'unité de la valeur
df_population['Population'] = df_population['Valeur']*1000

In [8]:
# Changer la type des données
df_population = df_population.astype({'Population':int})

In [9]:
# Sélection des colonnes utiles
df_population = df_population[['Zone', 'Année', 'Population']]
df_population.head()

,Zone,Année,Population
0,Afghanistan,2000,20779953
1,Afghanistan,2001,21606988
2,Afghanistan,2002,22600770
3,Afghanistan,2003,23680871
4,Afghanistan,2004,24726684


### Choisir une année de référence (2017)

In [10]:
# Sélection des données de l'année 2017
df_population_17 = df_population[df_population['Année'] == 2017].reset_index(drop=True) 

df_population_17

,Zone,Année,Population
0,Afghanistan,2017,36296113
1,Afrique du Sud,2017,57009756
2,Albanie,2017,2884169
3,Algérie,2017,41389189
4,Allemagne,2017,82658409
...,...,...,...
231,Venezuela (République bolivarienne du),2017,29402484
232,Viet Nam,2017,94600648
233,Yémen,2017,27834819
234,Zambie,2017,16853599


### Construire des variables: la croissance la population entre 2015 et 2017

In [11]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Zone        4411 non-null   object
 1   Année       4411 non-null   int64 
 2   Population  4411 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 103.5+ KB


In [12]:
# Changer le type des données
df_population = df_population.astype({'Année':str})

In [13]:
# Sélection des données de l'année 2015 et 2017 
popu_1= df_population.loc[df_population['Année'].isin(['2015','2017'])].reset_index(drop=True)
popu_1.head()

,Zone,Année,Population
0,Afghanistan,2015,34413603
1,Afghanistan,2017,36296113
2,Afrique du Sud,2015,55386367
3,Afrique du Sud,2017,57009756
4,Albanie,2015,2890513


In [14]:
#Pivot dataframe 
popu_evo = popu_1.pivot( index='Zone', columns='Année', values='Population')
popu_evo.head()

Année,2015,2017
Zone,,
Afghanistan,34413603,36296113
Afrique du Sud,55386367,57009756
Albanie,2890513,2884169
Algérie,39728025,41389189
Allemagne,81787411,82658409


In [15]:
# Ajouter la nouvelle variable: Evolution de la population de 2015-2017
popu_evo['Evo 2015_2017 %']=round((popu_evo['2017']-popu_evo['2015'])*100 /popu_evo['2015'],1)

In [16]:
# Suppression des colonnes inutiles
popu_evo.drop(columns = ["2015","2017"],inplace=True)

In [17]:
popu_evo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 236 entries, Afghanistan to Îles Wallis-et-Futuna
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Evo 2015_2017 %  236 non-null    float64
dtypes: float64(1)
memory usage: 3.7+ KB


### Jointure df_population_17 et popu_evo

In [18]:
df_population_17 = pd.merge(df_population_17, popu_evo, how='outer', on='Zone')

In [19]:
df_population_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 0 to 235
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Zone             236 non-null    object 
 1   Année            236 non-null    int64  
 2   Population       236 non-null    int64  
 3   Evo 2015_2017 %  236 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 9.2+ KB


In [20]:
df_population_17.head()

,Zone,Année,Population,Evo 2015_2017 %
0,Afghanistan,2017,36296113,5.5
1,Afrique du Sud,2017,57009756,2.9
2,Albanie,2017,2884169,-0.2
3,Algérie,2017,41389189,4.2
4,Allemagne,2017,82658409,1.1


In [21]:
# Sélection des colonnes utiles
df_population_17 = df_population_17[['Zone', 'Population', 'Evo 2015_2017 %']]

In [22]:
# Changer les noms des colonnes
df_population_17 = df_population_17.rename(columns={'Population': 'Population (2017)'})
df_population_17.head()

,Zone,Population (2017),Evo 2015_2017 %
0,Afghanistan,36296113,5.5
1,Afrique du Sud,57009756,2.9
2,Albanie,2884169,-0.2
3,Algérie,41389189,4.2
4,Allemagne,82658409,1.1


In [23]:
df_population_17['Zone'].nunique()

236

In [24]:
# Trouver le nombre de doublons
df_population_17.duplicated(['Zone']).sum()

0

In [25]:
# Nombre de valeurs manquantes
df_population_17.isnull().sum()

Zone                 0
Population (2017)    0
Evo 2015_2017 %      0
dtype: int64

## Données disponibilité alimentaire 2017

In [26]:
df_dispo_alim.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [27]:
df_dispo_alim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176600 entries, 0 to 176599
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Code Domaine            176600 non-null  object 
 1   Domaine                 176600 non-null  object 
 2   Code zone               176600 non-null  int64  
 3   Zone                    176600 non-null  object 
 4   Code Élément            176600 non-null  int64  
 5   Élément                 176600 non-null  object 
 6   Code Produit            176600 non-null  int64  
 7   Produit                 176600 non-null  object 
 8   Code année              176600 non-null  int64  
 9   Année                   176600 non-null  int64  
 10  Unité                   176600 non-null  object 
 11  Valeur                  176600 non-null  float64
 12  Symbole                 176600 non-null  object 
 13  Description du Symbole  176600 non-null  object 
dtypes: float64(1), int64

In [28]:
df_dispo_alim.describe()

,Code zone,Code Élément,Code Produit,Code année,Année,Valeur
count,176600.000000,176600.000000,176600.000000,176600.0,176600.0,176600.000000
mean,125.474740,3810.052894,2628.778545,2017.0,2017.0,210.554918
std,72.941699,2211.790089,91.993247,0.0,0.0,4762.047787
min,1.000000,645.000000,2511.000000,2017.0,2017.0,-10388.000000
25%,63.000000,684.000000,2556.000000,2017.0,2017.0,0.000000
50%,120.000000,5142.000000,2612.000000,2017.0,2017.0,0.120000
75%,188.000000,5301.000000,2732.000000,2017.0,2017.0,8.462500
max,276.000000,5911.000000,2899.000000,2017.0,2017.0,758548.000000


### Nettoyage

#### Créer dataframe: dispo_alim_total

In [29]:
# Sélection des colonnes utiles
df_dispo_alim = df_dispo_alim[['Zone', 'Élément', 'Produit', 'Unité','Valeur']]
df_dispo_alim.head()

,Zone,Élément,Produit,Unité,Valeur
0,Afghanistan,Production,Blé et produits,Milliers de tonnes,4281.0
1,Afghanistan,Importations - Quantité,Blé et produits,Milliers de tonnes,2302.0
2,Afghanistan,Variation de stock,Blé et produits,Milliers de tonnes,-119.0
3,Afghanistan,Exportations - Quantité,Blé et produits,Milliers de tonnes,0.0
4,Afghanistan,Disponibilité intérieure,Blé et produits,Milliers de tonnes,6701.0


In [30]:
# Verifier les valeurs uniques de la colonne 'Unité'
df_dispo_alim.Unité.unique()

array(['Milliers de tonnes', 'kg', 'Kcal/personne/jour',
       'g/personne/jour'], dtype=object)

In [31]:
# Changement de l'unité des valeurs en tonnes
#df_dispo_alim['Valeur']*= 1e3

In [32]:
#Pivot dataframe 'dispo_alim'
dispo_alim_total = df_dispo_alim.pivot_table(columns='Élément', index=['Zone'], values='Valeur',aggfunc=np.sum)
dispo_alim_total.head()

Élément,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
Zone,,,,,,,,,,,,,,,,,
Afghanistan,NaN,456.0,94.0,1997.0,357.31,30.63,54.09,15139.0,601.0,4555.0,12968.0,1146.0,11609.0,-111.0,470.0,115.0,426.0
Afrique du Sud,0.0,9371.0,1448.0,2987.0,556.42,81.92,83.36,66840.0,10968.0,9020.0,31726.0,3108.0,68455.0,-594.0,212.0,21564.0,-322.0
Albanie,NaN,697.0,177.0,3400.0,1260.49,117.57,119.50,4879.0,156.0,884.0,3631.0,321.0,4231.0,-115.0,53.0,113.0,72.0
Algérie,0.0,4884.0,2405.0,3345.0,798.71,79.62,92.85,45365.0,743.0,19718.0,33056.0,3683.0,26763.0,-22.0,358.0,993.0,374.0
Allemagne,NaN,30904.0,8316.0,3559.0,943.24,147.96,104.07,174960.0,58793.0,66550.0,77988.0,5756.0,167828.0,-831.0,1712.0,51109.0,651.0


In [33]:
# Sélection des colonnes utiles (unité milliers de tonnes)
dispo_alim_total = dispo_alim_total[['Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Nourriture']]
dispo_alim_total.head()

Élément,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture
Zone,,,,
Afghanistan,15139.0,601.0,4555.0,12968.0
Afrique du Sud,66840.0,10968.0,9020.0,31726.0
Albanie,4879.0,156.0,884.0,3631.0
Algérie,45365.0,743.0,19718.0,33056.0
Allemagne,174960.0,58793.0,66550.0,77988.0


In [34]:
dispo_alim_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, Afghanistan to Îles Salomon
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Disponibilité intérieure  174 non-null    float64
 1   Exportations - Quantité   174 non-null    float64
 2   Importations - Quantité   174 non-null    float64
 3   Nourriture                174 non-null    float64
dtypes: float64(4)
memory usage: 6.8+ KB


#### Créer dataframe: dispo_poulet

In [35]:
# Choisir uniquement les données des produits "Viande de Volailles"
dispo_poulet = df_dispo_alim[df_dispo_alim['Produit'] == 'Viande de Volailles'].reset_index(drop=True)
dispo_poulet.head(20)

,Zone,Élément,Produit,Unité,Valeur
0,Afghanistan,Production,Viande de Volailles,Milliers de tonnes,28.00
1,Afghanistan,Importations - Quantité,Viande de Volailles,Milliers de tonnes,29.00
2,Afghanistan,Variation de stock,Viande de Volailles,Milliers de tonnes,0.00
3,Afghanistan,Disponibilité intérieure,Viande de Volailles,Milliers de tonnes,57.00
4,Afghanistan,Pertes,Viande de Volailles,Milliers de tonnes,2.00
5,Afghanistan,Résidus,Viande de Volailles,Milliers de tonnes,0.00
6,Afghanistan,Nourriture,Viande de Volailles,Milliers de tonnes,55.00
7,Afghanistan,Disponibilité alimentaire en quantité (kg/pers...,Viande de Volailles,kg,1.53
8,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Viande de Volailles,Kcal/personne/jour,5.00
9,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Viande de Volailles,g/personne/jour,0.54


In [36]:
#Pivot dataframe 'dispo_poulet'
dispo_poulet = dispo_poulet.pivot(columns='Élément', index=['Zone'], values='Valeur')

In [37]:
dispo_poulet.head()

Élément,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
Zone,,,,,,,,,,,,,,,,,
Afghanistan,NaN,NaN,NaN,5.0,1.53,0.33,0.54,57.0,NaN,29.0,55.0,2.0,28.0,0.0,NaN,NaN,0.0
Afrique du Sud,0.0,NaN,NaN,143.0,35.69,9.25,14.11,2118.0,63.0,514.0,2035.0,83.0,1667.0,0.0,NaN,NaN,-0.0
Albanie,NaN,NaN,NaN,85.0,16.36,6.45,6.26,47.0,0.0,38.0,47.0,NaN,13.0,0.0,NaN,NaN,4.0
Algérie,0.0,NaN,NaN,22.0,6.38,1.50,1.97,277.0,0.0,2.0,264.0,13.0,275.0,0.0,NaN,NaN,0.0
Allemagne,NaN,NaN,NaN,71.0,19.47,4.16,7.96,1739.0,646.0,842.0,1609.0,NaN,1514.0,-38.0,NaN,167.0,-29.0


In [38]:
dispo_poulet.isna().sum()

Élément
Alimentation pour touristes                                       94
Aliments pour animaux                                            171
Autres utilisations (non alimentaire)                            138
Disponibilité alimentaire (Kcal/personne/jour)                     0
Disponibilité alimentaire en quantité (kg/personne/an)             0
Disponibilité de matière grasse en quantité (g/personne/jour)      0
Disponibilité de protéines en quantité (g/personne/jour)           0
Disponibilité intérieure                                           2
Exportations - Quantité                                           37
Importations - Quantité                                            2
Nourriture                                                         2
Pertes                                                           105
Production                                                         4
Résidus                                                            8
Semences                  

In [39]:
# Sélection des colonnes utiles
dispo_poulet = dispo_poulet[['Disponibilité intérieure', 'Exportations - Quantité',
                           'Importations - Quantité', 'Production', 'Nourriture']]

# Changer les noms des colonnes
dispo_poulet = dispo_poulet.rename(columns={'Disponibilité intérieure': 'Dispo intérieure poulet', 
                        'Exportations - Quantité': 'Exportations poulet',
                        'Importations - Quantité': 'Importations poulet', 
                        'Production': 'Production poulet','Nourriture': 'Consommation poulet'})

In [40]:
dispo_poulet.head()

Élément,Dispo intérieure poulet,Exportations poulet,Importations poulet,Production poulet,Consommation poulet
Zone,,,,,
Afghanistan,57.0,NaN,29.0,28.0,55.0
Afrique du Sud,2118.0,63.0,514.0,1667.0,2035.0
Albanie,47.0,0.0,38.0,13.0,47.0
Algérie,277.0,0.0,2.0,275.0,264.0
Allemagne,1739.0,646.0,842.0,1514.0,1609.0


In [41]:
dispo_poulet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172 entries, Afghanistan to Îles Salomon
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Dispo intérieure poulet  170 non-null    float64
 1   Exportations poulet      135 non-null    float64
 2   Importations poulet      170 non-null    float64
 3   Production poulet        168 non-null    float64
 4   Consommation poulet      170 non-null    float64
dtypes: float64(5)
memory usage: 8.1+ KB


In [42]:
dispo_poulet.isna().sum()

Élément
Dispo intérieure poulet     2
Exportations poulet        37
Importations poulet         2
Production poulet           4
Consommation poulet         2
dtype: int64

Il y a 37 valeurs d'exportations poulet est NAN, donc on considère que les valeurs sont nulls.

In [43]:
dispo_poulet['Exportations poulet'] = dispo_poulet['Exportations poulet'].fillna(0)

In [44]:
dispo_poulet.isna().sum()

Élément
Dispo intérieure poulet    2
Exportations poulet        0
Importations poulet        2
Production poulet          4
Consommation poulet        2
dtype: int64

In [45]:
dispo_poulet_nan_pro = dispo_poulet.loc[dispo_poulet['Production poulet'].isna()]
dispo_poulet_nan_pro

Élément,Dispo intérieure poulet,Exportations poulet,Importations poulet,Production poulet,Consommation poulet
Zone,,,,,
Djibouti,3.0,0.0,3.0,NaN,3.0
Maldives,12.0,0.0,12.0,NaN,7.0
Ouzbékistan,NaN,0.0,NaN,NaN,NaN
République démocratique populaire lao,NaN,0.0,NaN,NaN,NaN


In [46]:
dispo_poulet = dispo_poulet.loc[dispo_poulet['Dispo intérieure poulet'].notnull()]

In [47]:
dispo_poulet_nan_pro = dispo_poulet.loc[dispo_poulet['Production poulet'].isna()]
dispo_poulet_nan_pro

Élément,Dispo intérieure poulet,Exportations poulet,Importations poulet,Production poulet,Consommation poulet
Zone,,,,,
Djibouti,3.0,0.0,3.0,NaN,3.0
Maldives,12.0,0.0,12.0,NaN,7.0


In [48]:
dispo_poulet['Production poulet'] = dispo_poulet['Production poulet'].fillna(0)

In [49]:
dispo_poulet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170 entries, Afghanistan to Îles Salomon
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Dispo intérieure poulet  170 non-null    float64
 1   Exportations poulet      170 non-null    float64
 2   Importations poulet      170 non-null    float64
 3   Production poulet        170 non-null    float64
 4   Consommation poulet      170 non-null    float64
dtypes: float64(5)
memory usage: 8.0+ KB


#### Jointure df_population_17, dispo_alim_total et dispo_poulet

In [50]:
df_final0 = pd.merge(df_population_17, dispo_poulet, how='outer', on='Zone')

In [51]:
df_final1 = pd.merge(df_final0, dispo_alim_total, how='outer', on='Zone')

In [52]:
df_final1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 0 to 235
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Zone                      236 non-null    object 
 1   Population (2017)         236 non-null    int64  
 2   Evo 2015_2017 %           236 non-null    float64
 3   Dispo intérieure poulet   170 non-null    float64
 4   Exportations poulet       170 non-null    float64
 5   Importations poulet       170 non-null    float64
 6   Production poulet         170 non-null    float64
 7   Consommation poulet       170 non-null    float64
 8   Disponibilité intérieure  174 non-null    float64
 9   Exportations - Quantité   174 non-null    float64
 10  Importations - Quantité   174 non-null    float64
 11  Nourriture                174 non-null    float64
dtypes: float64(10), int64(1), object(1)
memory usage: 24.0+ KB


In [53]:
df_final_nan = df_final1.loc[df_final1['Production poulet'].isna()]
df_final_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 5 to 227
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Zone                      66 non-null     object 
 1   Population (2017)         66 non-null     int64  
 2   Evo 2015_2017 %           66 non-null     float64
 3   Dispo intérieure poulet   0 non-null      float64
 4   Exportations poulet       0 non-null      float64
 5   Importations poulet       0 non-null      float64
 6   Production poulet         0 non-null      float64
 7   Consommation poulet       0 non-null      float64
 8   Disponibilité intérieure  4 non-null      float64
 9   Exportations - Quantité   4 non-null      float64
 10  Importations - Quantité   4 non-null      float64
 11  Nourriture                4 non-null      float64
dtypes: float64(10), int64(1), object(1)
memory usage: 6.7+ KB


In [54]:
df_final1 = df_final1.loc[df_final1['Production poulet'].notnull()]

In [55]:
df_final1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 235
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Zone                      170 non-null    object 
 1   Population (2017)         170 non-null    int64  
 2   Evo 2015_2017 %           170 non-null    float64
 3   Dispo intérieure poulet   170 non-null    float64
 4   Exportations poulet       170 non-null    float64
 5   Importations poulet       170 non-null    float64
 6   Production poulet         170 non-null    float64
 7   Consommation poulet       170 non-null    float64
 8   Disponibilité intérieure  170 non-null    float64
 9   Exportations - Quantité   170 non-null    float64
 10  Importations - Quantité   170 non-null    float64
 11  Nourriture                170 non-null    float64
dtypes: float64(10), int64(1), object(1)
memory usage: 17.3+ KB


## Données d'Indicateur économique : Pouvoir d'achat par habitant (PIB/habitant)

In [56]:
df_economie.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,MK,Indicateurs macro,4,Afghanistan,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,520.616409,X,Ciffre de sources internationales,NaN
1,MK,Indicateurs macro,710,Afrique du Sud,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,6121.876572,X,Ciffre de sources internationales,NaN
2,MK,Indicateurs macro,8,Albanie,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,4514.204908,X,Ciffre de sources internationales,NaN
3,MK,Indicateurs macro,12,Algérie,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,4109.696001,X,Ciffre de sources internationales,NaN
4,MK,Indicateurs macro,276,Allemagne,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,44651.829102,X,Ciffre de sources internationales,NaN


In [57]:
df_economie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            211 non-null    object 
 1   Domaine                 211 non-null    object 
 2   Code zone (M49)         211 non-null    int64  
 3   Zone                    211 non-null    object 
 4   Code Élément            211 non-null    int64  
 5   Élément                 211 non-null    object 
 6   Code Produit            211 non-null    int64  
 7   Produit                 211 non-null    object 
 8   Code année              211 non-null    int64  
 9   Année                   211 non-null    int64  
 10  Unité                   211 non-null    object 
 11  Valeur                  211 non-null    float64
 12  Symbole                 211 non-null    object 
 13  Description du Symbole  211 non-null    object 
 14  Note                    0 non-null      fl

In [58]:
# Sélection des colonnes utiles
df_economie = df_economie[['Zone', 'Valeur']]

# Changer les noms des colonnes
df_economie = df_economie.rename(columns={'Valeur': 'PIB (US$)'})

### Jointure df_final1 et df_economie

In [59]:
df_final_eco = pd.merge(df_final1, df_economie, how='outer', on='Zone')

In [60]:
df_final_eco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Zone                      212 non-null    object 
 1   Population (2017)         170 non-null    float64
 2   Evo 2015_2017 %           170 non-null    float64
 3   Dispo intérieure poulet   170 non-null    float64
 4   Exportations poulet       170 non-null    float64
 5   Importations poulet       170 non-null    float64
 6   Production poulet         170 non-null    float64
 7   Consommation poulet       170 non-null    float64
 8   Disponibilité intérieure  170 non-null    float64
 9   Exportations - Quantité   170 non-null    float64
 10  Importations - Quantité   170 non-null    float64
 11  Nourriture                170 non-null    float64
 12  PIB (US$)                 211 non-null    float64
dtypes: float64(12), object(1)
memory usage: 23.2+ KB


In [61]:
#Afficher les intitulés uniques de la colonne 'Zone'
df_final_eco['Zone'].nunique()

212

In [62]:
#Afficher les intitulés uniques de la colonne 'Zone'
df_final_eco.Zone.unique()

array(['Afghanistan', 'Afrique du Sud', 'Albanie', 'Algérie', 'Allemagne',
       'Angola', 'Antigua-et-Barbuda', 'Arabie saoudite', 'Argentine',
       'Arménie', 'Australie', 'Autriche', 'Azerbaïdjan', 'Bahamas',
       'Bangladesh', 'Barbade', 'Bélarus', 'Belgique', 'Belize', 'Bénin',
       'Bolivie (État plurinational de)', 'Bosnie-Herzégovine',
       'Botswana', 'Brésil', 'Bulgarie', 'Burkina Faso', 'Cabo Verde',
       'Cambodge', 'Cameroun', 'Canada', 'Chili',
       'Chine - RAS de Hong-Kong', 'Chine - RAS de Macao',
       'Chine, continentale', 'Chine, Taiwan Province de', 'Chypre',
       'Colombie', 'Congo', 'Costa Rica', "Côte d'Ivoire", 'Croatie',
       'Cuba', 'Danemark', 'Djibouti', 'Dominique', 'Égypte',
       'El Salvador', 'Émirats arabes unis', 'Équateur', 'Espagne',
       'Estonie', 'Eswatini', "États-Unis d'Amérique", 'Éthiopie',
       'Fédération de Russie', 'Fidji', 'Finlande', 'France', 'Gabon',
       'Gambie', 'Géorgie', 'Ghana', 'Grèce', 'Grenade', 'Gu

In [63]:
df_final3_nan_pib = df_final_eco.loc[df_final_eco['PIB (US$)'].isna()]
df_final3_nan_pib

,Zone,Population (2017),Evo 2015_2017 %,Dispo intérieure poulet,Exportations poulet,Importations poulet,Production poulet,Consommation poulet,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,PIB (US$)
34,"Chine, Taiwan Province de",23674546.0,0.5,785.0,8.0,161.0,652.0,785.0,25365.0,1669.0,14429.0,15162.0,NaN


In [64]:
df_final_eco['PIB (US$)'].fillna(value=50.500, inplace=True)

In [65]:
df_final_eco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Zone                      212 non-null    object 
 1   Population (2017)         170 non-null    float64
 2   Evo 2015_2017 %           170 non-null    float64
 3   Dispo intérieure poulet   170 non-null    float64
 4   Exportations poulet       170 non-null    float64
 5   Importations poulet       170 non-null    float64
 6   Production poulet         170 non-null    float64
 7   Consommation poulet       170 non-null    float64
 8   Disponibilité intérieure  170 non-null    float64
 9   Exportations - Quantité   170 non-null    float64
 10  Importations - Quantité   170 non-null    float64
 11  Nourriture                170 non-null    float64
 12  PIB (US$)                 212 non-null    float64
dtypes: float64(12), object(1)
memory usage: 23.2+ KB


In [66]:
df_final_eco = df_final_eco.loc[df_final_eco['Nourriture'].notnull()]

In [67]:
df_final_eco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Zone                      170 non-null    object 
 1   Population (2017)         170 non-null    float64
 2   Evo 2015_2017 %           170 non-null    float64
 3   Dispo intérieure poulet   170 non-null    float64
 4   Exportations poulet       170 non-null    float64
 5   Importations poulet       170 non-null    float64
 6   Production poulet         170 non-null    float64
 7   Consommation poulet       170 non-null    float64
 8   Disponibilité intérieure  170 non-null    float64
 9   Exportations - Quantité   170 non-null    float64
 10  Importations - Quantité   170 non-null    float64
 11  Nourriture                170 non-null    float64
 12  PIB (US$)                 170 non-null    float64
dtypes: float64(12), object(1)
memory usage: 18.6+ KB


In [68]:
df_final_eco

,Zone,Population (2017),Evo 2015_2017 %,Dispo intérieure poulet,Exportations poulet,Importations poulet,Production poulet,Consommation poulet,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,PIB (US$)
0,Afghanistan,36296113.0,5.5,57.0,0.0,29.0,28.0,55.0,15139.0,601.0,4555.0,12968.0,520.616409
1,Afrique du Sud,57009756.0,2.9,2118.0,63.0,514.0,1667.0,2035.0,66840.0,10968.0,9020.0,31726.0,6121.876572
2,Albanie,2884169.0,-0.2,47.0,0.0,38.0,13.0,47.0,4879.0,156.0,884.0,3631.0,4514.204908
3,Algérie,41389189.0,4.2,277.0,0.0,2.0,275.0,264.0,45365.0,743.0,19718.0,33056.0,4109.696001
4,Allemagne,82658409.0,1.1,1739.0,646.0,842.0,1514.0,1609.0,174960.0,58793.0,66550.0,77988.0,44651.829102
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Venezuela (République bolivarienne du),29402484.0,-2.3,626.0,0.0,25.0,600.0,596.0,21360.0,71.0,4662.0,15699.0,8432.288335
166,Viet Nam,94600648.0,2.1,1206.0,1.0,291.0,918.0,1166.0,119375.0,24182.0,25602.0,67490.0,2365.521494
167,Yémen,27834819.0,5.0,246.0,0.0,78.0,168.0,237.0,8926.0,277.0,6131.0,8415.0,745.087677
168,Zambie,16853599.0,6.1,60.0,1.0,12.0,49.0,58.0,12219.0,831.0,527.0,5998.0,1534.874627


## Enlever les colonnes

In [69]:
df_final_short = df_final_eco.drop(
    columns=['Nourriture','Exportations - Quantité', 'Importations - Quantité', 'Disponibilité intérieure'])

## Ajouter les colonnes

In [70]:
df_final_eco['Taux_import_poulet %'] = round((df_final_eco['Importations poulet']*100/ df_final_eco['Importations - Quantité']),2)

In [71]:
df_final_eco['Taux_export_poulet %'] = round((df_final_eco['Exportations poulet']*100/ df_final_eco['Exportations - Quantité']),2)

In [72]:
df_final_eco['Taux_conso_poulet %'] = round((df_final_eco['Consommation poulet']*100/ df_final_eco['Nourriture']),2)

In [73]:
df_final_eco['Taux_dispo_intér_poulet %'] = round((df_final_eco['Dispo intérieure poulet']*100/ df_final_eco['Disponibilité intérieure']),2)

In [74]:
# Suppression des colonnes inutiles
df_final_eco = df_final_eco.drop(
    columns=['Nourriture','Exportations - Quantité', 'Importations - Quantité', 'Disponibilité intérieure'])

In [75]:
df_final_eco['Taux_export_poulet %'] = df_final_eco['Taux_export_poulet %'].fillna(0)

In [76]:
df_final_eco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Zone                       170 non-null    object 
 1   Population (2017)          170 non-null    float64
 2   Evo 2015_2017 %            170 non-null    float64
 3   Dispo intérieure poulet    170 non-null    float64
 4   Exportations poulet        170 non-null    float64
 5   Importations poulet        170 non-null    float64
 6   Production poulet          170 non-null    float64
 7   Consommation poulet        170 non-null    float64
 8   PIB (US$)                  170 non-null    float64
 9   Taux_import_poulet %       170 non-null    float64
 10  Taux_export_poulet %       170 non-null    float64
 11  Taux_conso_poulet %        170 non-null    float64
 12  Taux_dispo_intér_poulet %  170 non-null    float64
dtypes: float64(12), object(1)
memory usage: 18.6+ KB


In [77]:
df_economy.head()

,Pays,Economy 2019,Economy 2020
0,Afghanistan,44.2,44.1
1,Afrique du Sud,66.7,67.0
2,Albanie,67.0,67.7
3,Algérie,48.5,48.6
4,Allemagne,79.3,79.7


In [78]:
df_economy = df_economy.rename(columns={'Pays': 'Zone', 'Economy 2020': 'Facilité de faire des affaires 2020'})

In [79]:
df_economy = df_economy.drop('Economy 2019', 1)

In [80]:
df_final_economy = pd.merge(df_economy, df_final_eco, how='outer', on='Zone')

In [81]:
df_final_economy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 0 to 219
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Zone                                 220 non-null    object 
 1   Facilité de faire des affaires 2020  220 non-null    float64
 2   Population (2017)                    170 non-null    float64
 3   Evo 2015_2017 %                      170 non-null    float64
 4   Dispo intérieure poulet              170 non-null    float64
 5   Exportations poulet                  170 non-null    float64
 6   Importations poulet                  170 non-null    float64
 7   Production poulet                    170 non-null    float64
 8   Consommation poulet                  170 non-null    float64
 9   PIB (US$)                            170 non-null    float64
 10  Taux_import_poulet %                 170 non-null    float64
 11  Taux_export_poulet %            

In [82]:
df_final_economy_nan = df_final_economy.loc[df_final_economy['Population (2017)'].isna()]
df_final_economy_nan

,Zone,Facilité de faire des affaires 2020,Population (2017),Evo 2015_2017 %,Dispo intérieure poulet,Exportations poulet,Importations poulet,Production poulet,Consommation poulet,PIB (US$),Taux_import_poulet %,Taux_export_poulet %,Taux_conso_poulet %,Taux_dispo_intér_poulet %
14,Bahreïn,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Bangladesh - Chittagong,45.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Bangladesh - Dacca,44.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Bhoutan,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Brésil - Rio de Janeiro,59.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,Brésil - São Paulo,58.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Brunéi Darussalam,70.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,Burundi,46.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,Chine,77.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Chine - Shanghai,77.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
#Afficher les intitulés uniques de la colonne 'Zone'
#df_final_economy.Zone.unique()

In [84]:
df_final_economy = df_final_economy.loc[df_final_economy['Population (2017)'].notnull()]

In [85]:
df_final_economy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 219
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Zone                                 170 non-null    object 
 1   Facilité de faire des affaires 2020  170 non-null    float64
 2   Population (2017)                    170 non-null    float64
 3   Evo 2015_2017 %                      170 non-null    float64
 4   Dispo intérieure poulet              170 non-null    float64
 5   Exportations poulet                  170 non-null    float64
 6   Importations poulet                  170 non-null    float64
 7   Production poulet                    170 non-null    float64
 8   Consommation poulet                  170 non-null    float64
 9   PIB (US$)                            170 non-null    float64
 10  Taux_import_poulet %                 170 non-null    float64
 11  Taux_export_poulet %            

# Données stabilité politique et tax douane

In [86]:
df_stabilite.head()

,Zone,Index de stabilité,Taux des droits de douane
0,Afghanistan,-2.80,5.63
1,Angola,-0.33,9.23
2,Albanie,0.38,1.10
3,Émirats arabes unis,0.62,3.28
4,Argentine,0.17,6.88


In [87]:
df_stabilite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Zone                       154 non-null    object 
 1   Index de stabilité         154 non-null    float64
 2   Taux des droits de douane  154 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.7+ KB


In [88]:
df_final_stabilite = pd.merge(df_final_economy, df_stabilite, how='inner', on='Zone')

In [89]:
df_final_stabilite.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 0 to 153
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Zone                                 154 non-null    object 
 1   Facilité de faire des affaires 2020  154 non-null    float64
 2   Population (2017)                    154 non-null    float64
 3   Evo 2015_2017 %                      154 non-null    float64
 4   Dispo intérieure poulet              154 non-null    float64
 5   Exportations poulet                  154 non-null    float64
 6   Importations poulet                  154 non-null    float64
 7   Production poulet                    154 non-null    float64
 8   Consommation poulet                  154 non-null    float64
 9   PIB (US$)                            154 non-null    float64
 10  Taux_import_poulet %                 154 non-null    float64
 11  Taux_export_poulet %            

# Conlusion

In [90]:
df_final = df_final_stabilite

In [91]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 0 to 153
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Zone                                 154 non-null    object 
 1   Facilité de faire des affaires 2020  154 non-null    float64
 2   Population (2017)                    154 non-null    float64
 3   Evo 2015_2017 %                      154 non-null    float64
 4   Dispo intérieure poulet              154 non-null    float64
 5   Exportations poulet                  154 non-null    float64
 6   Importations poulet                  154 non-null    float64
 7   Production poulet                    154 non-null    float64
 8   Consommation poulet                  154 non-null    float64
 9   PIB (US$)                            154 non-null    float64
 10  Taux_import_poulet %                 154 non-null    float64
 11  Taux_export_poulet %            

In [92]:
df_final

,Zone,Facilité de faire des affaires 2020,Population (2017),Evo 2015_2017 %,Dispo intérieure poulet,Exportations poulet,Importations poulet,Production poulet,Consommation poulet,PIB (US$),Taux_import_poulet %,Taux_export_poulet %,Taux_conso_poulet %,Taux_dispo_intér_poulet %,Index de stabilité,Taux des droits de douane
0,Afghanistan,44.1,36296113.0,5.5,57.0,0.0,29.0,28.0,55.0,520.616409,0.64,0.00,0.42,0.38,-2.80,5.63
1,Afrique du Sud,67.0,57009756.0,2.9,2118.0,63.0,514.0,1667.0,2035.0,6121.876572,5.70,0.57,6.41,3.17,-0.28,4.40
2,Albanie,67.7,2884169.0,-0.2,47.0,0.0,38.0,13.0,47.0,4514.204908,4.30,0.00,1.29,0.96,0.38,1.10
3,Algérie,48.6,41389189.0,4.2,277.0,0.0,2.0,275.0,264.0,4109.696001,0.01,0.00,0.80,0.61,-0.92,10.16
4,Allemagne,79.7,82658409.0,1.1,1739.0,646.0,842.0,1514.0,1609.0,44651.829102,1.27,1.10,2.06,0.99,0.59,1.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,Zambie,66.9,16853599.0,6.1,60.0,1.0,12.0,49.0,58.0,1534.874627,2.28,0.12,0.97,0.49,0.15,4.82
150,Zimbabwe,54.5,14236595.0,3.1,76.0,0.0,6.0,69.0,67.0,1548.186368,0.56,0.00,1.39,0.83,-0.71,4.99
151,République de Corée,5.0,51096415.0,0.5,854.0,6.0,137.0,838.0,854.0,31781.123320,0.47,0.18,2.09,1.40,0.32,5.48
152,Tchéquie,0.0,10641034.0,0.4,249.0,27.0,116.0,163.0,234.0,20545.836143,1.81,0.30,2.53,1.21,1.00,1.48


In [93]:
df_final.to_csv('df_final.csv', index=False) 